# Programming Assingnment 5.1 - Text Generation and Segmentation with Hugging Face

안녕하세요, **AIKU 학회원 여러분**. 어느덧 마지막 과제가 주어지게 되었네요. 그동안 강의를 듣고 과제를 하느라 정말 고생 많으셨습니다. 이제 거의 마무리 단계이니, 조금만 더 힘내셔서 끝까지 잘 마무리해주시면 감사하겠습니다 😊

다섯 번째 과제에서는 Hugging Face 라이브러리를 사용하여 모델을 불러와 학습 및 추론을 진행할 예정입니다. NLP와 CV 분야에서 각각 2개의 모델을 사용하며, 수행할 작업은 **텍스트 생성(Text Generation), 이미지 분할(Segmentation), LLM Agent 구축, 이미지 생성 (Image Generation)** 입니다. 이전 과제들과는 달리, 사전 학습된 모델을 불러오기 때문에 모델 구조를 직접 구현할 필요가 없습니다. 대신, Hugging Face에서 제공하는 다양한 클래스와 함수를 활용하게 됩니다.

모델을 직접 구현하고 학습 및 추론하는 과정은 특히 연구 측면에서 매우 중요하지만, 이미 개발된 라이브러리를 효과적으로 활용하는 능력 또한 중요합니다. 이는 연구는 물론 대부분의 AI 프로젝트, 그리고 Kaggle이나 데이콘 같은 데이터 경진 대회에서도 필수적인 능력입니다.

이번 과제과제는 여러분이 Hugging Face에 익숙해지도록 설계되었습니다. 이를 통해 Hugging Face를 능숙하게 다루는 실력을 키울 수 있기를 바랍니다.

![](https://miro.medium.com/v2/resize:fit:1000/1*09M-TxFBwUVSteNYAiIcpg.png)

## Part 1. Text Generation with KoGPT

### 1-1. Pip install & Imports
필요한 패키지를 pip를 이용해서 설치합니다.

In [ ]:
!pip install transformers accelerate datasets tqdm

필요한 라이브러리를 import하고, device를 GPU로 바꿔줍니다.

다만, 코드를 구현하는 과정에서는 **CPU 사용**을 권장드립니다.

GPU가 가장 많이 필요한 단계는 학습하는 과정인데 구현하는 시간동안 colab의 GPU를 다 써버리면 막상 학습할 때 필요한 GPU 자원을 쓸 수 없게 됩니다. 따라서 코드를 모두 구현하고 train 코드가 잘 돌아가는지 확인한 뒤에 colab 상단 런타임 메뉴에서 런타임 유형을 GPU로 바꾼 뒤에 실행하면 됩니다.

In [ ]:
import os
import random

import torch
from torch import nn, Tensor

from transformers import (
    PreTrainedTokenizerFast,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    GPT2LMHeadModel
)

from datasets import load_dataset, Dataset, DatasetDict

from collections import defaultdict
from tqdm import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device: ', device)

### 1-2. Prepare Dataset

아래 코드를 실행해서 데이터셋을 불러와 주세요. 여러분들께서 사용하실 데이터셋은
**[한국어 일상 속 공감형 대화 데이터셋](https://huggingface.co/datasets/Smoked-Salmon-s/empathetic_dialogues_ko)** 입니다.

일상 속 다양한 상황에서 사용자와 챗봇 간의 대화를 담은 데이터셋이고 GPT4, GPT3.5-turbo로 제작된 데이터입니다.

답변은 [공감적 표현 - 일반적인 대화 - 관련된 질문] 의 형태를 가집니다.

해당 데이터셋은 싱글-턴, 2-턴, 3-턴 대화로 구성되어 있습니다. 해당 과제에서는 싱글-턴 데이터만 활용합니다. 아래 `filter_dataset` 함수를 이용해 싱글-턴 데이터만으로 구성되어 있도록 필터링을 하며, 이는 Hugging Face에서 데이터셋을 불러올 때 무조건 해야 하는 과정이 아닌, 특정 조건에 따라 특정 데이터만을 가져오고 싶을 때 사용하는 코드입니다.

In [ ]:
def filter_dataset(dataset):
    filtered_list = []
    total = 0
    for sample in iter(dataset):
        total += 1
        if sample['type'] == 'single':
              filtered_list.append({'instruction': sample['instruction'], 'output': sample['output']})
    print(f"{len(filtered_list)/total:.2%} of data after filtering.")
    return Dataset.from_list(filtered_list)

In [ ]:
datasets = load_dataset("Smoked-Salmon-s/empathetic_dialogues_ko", split="train")
datasets = filter_dataset(datasets)   # filtering을 했을 때와 하지 않았을 때의 차이가 궁금하다면 해당 코드를 주석 처리하고 실행해보세요!

데이터셋의 구조를 한번 확인해보세요:

In [ ]:
datasets

해당 데이터셋은 training set으로만 구성이 되어 있어, Hugging Face의 `train_test_split` 함수를 활용해서 test set을 만들어봅시다.

In [ ]:
train_test_split = datasets.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

만들어진 training set과 test set을 `DatasetDict`를 활용해서 `dictionary` 형태로 만들어봐요. 이와 같이 하는 이유는 조금 뒤에 알려드리겠습니다.

In [ ]:
conversation_datasets = DatasetDict(
    {
        "train": train_dataset,
        "test": test_dataset,
    }
)

`DatasetDict`로 만들어진 데이터셋의 형태는 다음과 같습니다:

In [ ]:
conversation_datasets

이번에는 실제로 데이터가 어떻게 생겼는지 확인해봅시다:

In [ ]:
for i, data in enumerate(conversation_datasets["train"]):
  if i == 5:
    break
  for key, value in data.items():
     print(f"{key}: {value}")
  print("\n\n")

`transformers`의 `PreTrainedTokenizerFast`를 활용해서 tokenizer를 불러와 주세요. 모델 이름은 `skt/kogpt2-base-v2"`, `bos_token`은 `</s>`, `eos_token`은 `</s>`, `unk_token`은 `<unk>`, `pad_token`은 `<pad>`, `mask_token`은 `<mask>` 입니다.

다음 문서를 참고해주세요:
[PreTrainedTokenizerBase
](https://huggingface.co/docs/transformers/v4.44.0/en/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.from_pretrained)

In [ ]:
# TODO: tokenizer 구현
tokenizer =

만들어진 training set과 test set을 `DatasetDict`를 활용해서 `dictionary` 형태로 만든 이유는 아래 코드에 있는 [DatasetDict.map](https://huggingface.co/docs/datasets/v2.20.0/en/package_reference/main_classes#datasets.DatasetDict.map)을 활용하기 위해서입니다. Python의 `map` 함수와 같이 `DatasetDict.map` 함수를 사용하면 데이터셋에 있는 각각의 데이터 샘플에 대해 특정 함수를 적용하여 전처리를 할 수 있습니다. 예시를 하나 보여드리겠습니다:

```
from datasets import load_dataset
ds = load_dataset("rotten_tomatoes")
def add_prefix(example):
    example["text"] = "Review: " + example["text"]
    return example
ds = ds.map(add_prefix)

ds["train"][0:3]["text"]
#['Review: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
# 'Review: the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .',
# 'Review: effective but too-tepid biopic']

```
위 함수에서는 `map`함수를 사용하여 각각의 데이터 샘플에 대해 'Review'라는 prefix를 추가합니다.


---


과제에서는 `tokenize`라는 함수를 별도로 구현하고 `DatasetDict.map`을 사용하여 데이터 전처리를 진행합니다.
 `tokenize`함수에서 데이터가 모델에 입력으로 들어갈 수 있도록 해야하는 일은
1. 자연어인 token을 token id로 변환(encoding)
2. `instruction` 과 `output`을 구분할 token도 추가
3. 데이터셋 key값을 `input_ids`로 변환

이렇게 3가지입니다.

2번에서 `instruction`과 `output`을 구분할 token을 추가하는 이유는 아래 그림에 나온 예시를 이용해 설명드리겠습니다
![](https://jalammar.github.io/images/gpt2/decoder-only-transformer-translation.png)
![](https://jalammar.github.io/images/gpt2/decoder-only-summarization.png)
기계번역 task에서 영어-프랑스어 번역이라 했을 때 영어 문장과 대응되는 프랑스어 문장를 구분할 `<to-fr>` token을 추가하고, 요약 task에서는 요약 대상(article)과 요약본(summary)을 구분할 `<summarize>` token을 추가합니다. 만약 이를 구분할 token이 없었다면, 어디까지가 영어 문장인지, 어디까지가 요약 대상인지 모델이 구분하지 못합니다. 따라서 서로 다른 성격을 가진 text가 등장할 때, 중간에 token을 추가하는 방식으로 구분하곤 합니다.

3번에서 데이터셋 key값을 `input_ids`로 변환하는 이유는 Hugging Face에서 데이터 처리 인터페이스와 관련이 있습니다. 해당 내용은 조금 뒤에 알려드리겠습니다.

In [ ]:
def tokenize(data):
    # 질문, 레이블 응답 문장을 불러옵니다.
    instruction, output = data['instruction'], data['output']

    # 문장의 BOS token : </s>
    bos_token = tokenizer.bos_token
    # 문장의 EOS token : </s>
    eos_token = tokenizer.eos_token

    sentence = tokenizer.encode(
        # TODO : 데이터를 "BOS + 입력 + <unused0>(토큰으로 구분) + 출력 + EOS" 포맷으로 변환
        text=
        truncation=True,
        max_length=128,
        padding="max_length",
        )

    data['input_ids'] = sentence

    return data


tokenized_datasets = conversation_datasets.map(
    tokenize, remove_columns=conversation_datasets["train"].column_names
)

### 1-3. Train

#### 1-3-1. Load Model

사용할 모델을 불러오고 parameter 개수도 한번 확인해보세요

In [ ]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)
model_size = sum(t.numel() for t in model.parameters())
print(f"KoGPT-2 size: {model_size/1000**2:.1f}M parameters")

모델 내부 구조도 한번 확인해보세요.

In [ ]:
model

위의 출력 결과를 기반으로 KoGPT의 모델 구조에 대해 간단하게 설명드리겠습니다.

![](https://jalammar.github.io/images/xlnet/transformer-decoder-intro.png)
#### 1. **Main Components**
- **`GPT2LMHeadModel`**: GPT-2 기본 transformer model과 linear language model head가 결합된 최상위 모듈로, sequence의 다음 token을 예측하는 데 사용됩니다.

#### 2. **Transformer Model**
- **`transformer`: GPT2Model**
  - **`wte`: Embedding(51200, 768)**: Token embedding matrix로 input token의 id값을 이용해 768-dimensional vector로 매핑합니다. 어휘 크기는 51,200이고 이는 고유한 token (id) 값이 51200개라는 의미입니다.
  - **`wpe`: Embedding(1024, 768)**: Position embedding matrix로 최대 길이는 1024이고 token embedding과 마찬가지로 768-dimensional vector로 매핑합니다.
  - **`drop`: Dropout(p=0.1, inplace=False)**: Dropout layer로 임의의 값을 training때 0으로 설정해 overfitting을 방지합니다.
#### 3. **Decoder Blocks**
- **`h`: ModuleList**
  - 12개의 동일한 레이어(**`GPT2Block`**)가 포함되어 있으며, 각 레이어는 다음의 하위 모듈을 포함합니다:
    - **`ln_1`: LayerNorm**: Layer normalization
    - **`attn`: GPT2SdpaAttention**
      - **`c_attn`: Conv1D()**: 입력값을 query, key, value값으로 project해주는 Convolution layer
      - **`c_proj`: Conv1D()**: Masked Self-Attention의 output을 embedding의 feature space로 다시 project 해주는 Convolution layer
      - **`attn_dropout`: Dropout(p=0.1, inplace=False)**: Dropout
      - **`resid_dropout`: Dropout(p=0.1, inplace=False)**: Dropout
    - **`ln_2`: LayerNorm**: Layer Normalization
    - **`mlp`: GPT2MLP**
      - **`c_fc`: Conv1D()**: 768 dimension에서 더 높은 중간 차원(일반적으로 4배 큰)으로 차원을 확장하는 Convolution layer입니다.
      - **`c_proj`: Conv1D()**: `c_fc` layer의 output을 다시 768 dimension으로 project해주는 Convolution layer입니다.
      - **`act`: NewGELUActivation()**: GELU activation function
      - **`dropout`: Dropout(p=0.1, inplace=False)**: Dropout

#### 4. **Output Normalization**
- **`ln_f`: LayerNorm((768,), eps=1e-05, elementwise_affine=True)**: Layer Normalization

#### 5. **Language Model Head**
- **`lm_head`: Linear(in_features=768, out_features=51200, bias=False)**: A linear layer used to map the output of the transformer to logits corresponding to the vocabulary size. This is where the model predicts the next token in the sequence. 최종 output을 vocabulary size에 맞춰 logit으로 변환해주는 linear layer입니다. 모델이 sequence의 다음 token을 예측하는 위치입니다.

**갑자기 `Conv1D()`가 등장해서 당황하실 수 있는데, 새로운건 없고 여러분들께서 배우신 self-attention mechanism에서 query, key, value값을 만드는 과정에 필요한 layer라고 이해하시면 됩니다. Convolution layer를 활용했을 뿐 본질은 달라지지 않습니다!**

설명이 부족하다면 다음 자료를 참고하셔도 좋고 질문도 언제든지 환영입니다!

[Illustrated GPT2](https://jalammar.github.io/illustrated-gpt2/)



### 1-3-2. Data Collator

[DataCollatorForLanguageModeling](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForLanguageModeling) 를 사용해 `data_collator`를 정의합니다.

이름이 익숙하지 않나요? 과제3에서 서술형 문제중에

```
`DataLoader`에서 `collate_fn` parameter의 역할이랑 아래에 정의된 `collate_batch` 함수에 대해 설명해주세요:
```
이라는 문제가 있었습니다.

간단하게 설명을 하자면 [`DataLoader`에서 `collate_fn`은 데이터를 batch 단위로 처리하기 위해 사용됩니다.](https://pytorch.org/docs/stable/data.html#dataloader-collate-fn). Pytorch에서 `DataLoader`의 `collate_fn`을 사용하는 것처럼 Hugging Face에서는 `Data Collator`를 사용할 수 있고, 과제에서는 Language Modeling에 적용되는 `DataCollatorForLanguageModeling`을 사용하게 됩니다.

General한 `DataLoader`의 `collate_fn`과 달리, Hugging Face의 `Data Collator` class는 NLP에 특화되어 있고 이미 구현되어 있는 여러 method가 있다는 점에서 차이가 있습니다.


---



위에서 데이터셋 key값을 `input_ids`로 변환한 이유는 `DataCollatorForLanguageModeling`을 사용하기 위해서입니다.

`out = data_collator([tokenized_datasets["train"][i] for i in range(5)])` 이 코드가 실행될 때 실제로는 아래 함수가 실행됩니다.
```
    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        # Handle dict or lists with proper padding and conversion to tensor.
        if isinstance(examples[0], Mapping):
            batch = pad_without_fast_tokenizer_warning(
                self.tokenizer, examples, return_tensors="pt", pad_to_multiple_of=self.pad_to_multiple_of
            )
        else:
            batch = {
                "input_ids": _torch_collate_batch(examples, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)
            }

        # If special token mask has been preprocessed, pop it from the dict.
        special_tokens_mask = batch.pop("special_tokens_mask", None)
        if self.mlm:
            batch["input_ids"], batch["labels"] = self.torch_mask_tokens(
                batch["input_ids"], special_tokens_mask=special_tokens_mask
            )
        else:
            labels = batch["input_ids"].clone()
            if self.tokenizer.pad_token_id is not None:
                labels[labels == self.tokenizer.pad_token_id] = -100
            batch["labels"] = labels
        return batch
```
즉, 데이터에 key값으로 `input_ids`가 있어야만 실행됩니다. 코드가 워낙 길어서 일부분만 가져왔는데, 추가적인 설명을 원하시면 말씀해주세요!

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

### 1-3-3. Trainer

그동안의 과제에서는 여러분들께서 직접 Pytorch를 활용해 학습을 위한 코드를 구현해왔습니다. Hugging Face에서는 Pytorch에서처럼 학습을 할 수 있게 도와주는 [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer) class가 있습니다. 자주 사용하는 클래스이니 해당 링크에 들어가서 간단하게라도 한번 읽어보시는걸 추천드립니다. 통상적으로 다음과 같은 구조를 많이 사용합니다(**hyperparameter 값들은 임의로 설정했습니다. 각각의 parameter가 아닌 전체적인 구조에 집중해주세요**):
```
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="codeparrot-ds",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)
trainer.train()
```
보통 `TrainingArguments`에서는 hyperparameter를 비롯해 training과 관련된 configuration을 정의합니다. 그리고 `Trainer`에는 학습에 사용될 model, tokenizer, data collator, train dataset, eval dataset, `TrainingArguments`로 정의한 변수를 인자로 전달합니다.

과제에서 구현할 `Trainer`에서는 위에서 정의한 `model`, `tokenizer`, `data_collator`, `tokenized_datasets["train"]`, `tokenized_datasets["test"]`를 사용하면 되고 `TrainingArguments`로 정의한 `args`도 입력값으로 넣어주세요.

**Training loss는 2 이하로 떨어져야 통과입니다! 학습은 10-20분 정도 걸릴 예정입니다.**

In [ ]:
# TODO: TrainingArguments, Trainer 이용해서 학습
args =
trainer =

### 1-4. Inference

예시 5개를 만들어서 직접 결과를 출력해보세요. 학습 때와 마찬가지로, 입력과 출력을 구분짓기 위해 `<unused0>` token을 삽입해야 합니다.

In [ ]:
text_1 = ''
text_2 = ''
text_3 = ''
text_4 = ''
text_5 = ''
textset = [text_1, text_2, text_3, text_4, text_5]

for text in textset:
  # TODO: '<unused0>' token 삽입
  sentence =
  input_ids = tokenizer.encode(sentence, return_tensors='pt').to(device)
  gen_ids = model.generate(input_ids,
                            max_length=128,
                            repetition_penalty=2.0,
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            use_cache=True)
  generated = tokenizer.decode(gen_ids[0])
  print(f"Input: {text}\nOutput: {generated}\n\n")

`tokenizer.encode()` 대신 `tokenizer()`를 사용해보세요.

In [ ]:
for text in textset:
  sentence = text + '<unused0>'
  input_ids = tokenizer(sentence, return_tensors='pt').to(device)

  # TODO: input_ids 값 전달
  gen_ids = model.generate(
                            max_length=128,
                            repetition_penalty=2.0,
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            use_cache=True)
  generated = tokenizer.decode(gen_ids[0])
  print(f"Input: {text}\nOutput: {generated}\n\n")

**`tokenizer.encode()`와 `tokenizer()`의 차이, 그리고 이에 따라 왜 `model.generate()`의 입력값이 변하게 되는지 그 이유를 작성해주세요:**

# Part 2 Segmentation
이번 파트에서는 Transformer 기반의 Segmentation 모델인 MaskFormer를, HuggingFace를 활용해 학습해볼 것입니다.

## 2-1. Pip install

In [ ]:
# 필요한 패키지를 설치합니다
# transfomers: huggingface의 모델을 가져오도록 도와줍니다.
# datasets: huggingface의 데이터셋을 가져오도록 도와줍니다. (에러가 날 수 있으나 무시하고 진행하시면 됩니다.)
# evaluate: huggingface의 evaluating 관련 라이브러리를 가져오도록 도와줍니다.
# albumentations: 이미지 데이터의 data augmentation을 도와줍니다.

!pip install --upgrade pip
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets
!pip install -q evaluate
!pip install -q albumentations

## 2-2. Hugging Face Login
허깅페이스의 자원을 이용하기 위해 로그인합니다.

https://huggingface.co/

허깅페이스 사이트에 들어가서 회원가입 및 로그인을 하고,

우측 상단 프로필 -> Settings -> Access Tokens -> Create new token -> Read token을 발급하고,

아래 셀을 실행시킨 뒤 발급된 키를 넣으면 되겠습니다!

In [ ]:
# huggingface에 접근하기 위해 access token으로 로그인합니다.
import huggingface_hub
huggingface_hub.login()

## 2-3. Load Dataset
허깅페이스에서 segmentation 데이터셋을 다운로드 받고, 어떻게 생겼는지 확인해봅시다.

저희가 사용할 데이터셋은 sidewalk-semantic 입니다.
&nbsp;

&nbsp;

해당 데이터셋은 인도를 촬영한 이미지의 각 픽셀을 총 35개의 레이블로 annotation 해놓은, semantic segmentation 데이터셋입니다.


데이터셋이 다운되지 않는다면, 아래 링크로 가서 승인 버튼을 눌러주세요!


https://huggingface.co/datasets/segments/sidewalk-semantic

&nbsp;

데이터셋의 구성은 매우 간단합니다!

`['pixel_values']`에는 원본 이미지가, `['label']`에는 semantic segmentation을 위한 segmentation map이 들어있습니다!

In [ ]:
# 학습 데이터셋을 다운로드합니다.
from datasets import load_dataset
dataset = load_dataset("segments/sidewalk-semantic")

In [ ]:
dataset

In [ ]:
# 데이터셋을 섞고 train과 test를 나눕니다.
dataset = dataset.shuffle(seed=1)
dataset = dataset["train"].train_test_split(test_size=0.2)
train_ds = dataset["train"]
test_ds = dataset["test"]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 데이터가 어떻게 생겨먹었는지 한 번 볼까요?
example = train_ds[0]
image = example['pixel_values']
segmentation_map = np.array(example['label'])

# 원본 이미지를 봅시다.
plt.axis('off')
plt.imshow(image)
plt.show(image)

# Segmentation map을 봅시다.
plt.axis('off')
plt.imshow(segmentation_map)
plt.show()

In [ ]:
# Class Label을 색깔로 매핑하기 위한 팔레트입니다.
def color_palette():
    """Color palette that maps each class to RGB values.

    This one is actually taken from ADE20k.
    """
    return [[120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50],
            [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255],
            [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7],
            [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82],
            [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3],
            [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255],
            [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220],
            [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224],
            [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255],
            [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7],
            [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153],
            [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255],
            [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0],
            [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255],
            [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255],
            [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255],
            [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0],
            [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0],
            [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255],
            [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255],
            [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20],
            [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255],
            [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255],
            [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255],
            [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0],
            [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0],
            [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255],
            [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112],
            [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160],
            [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163],
            [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0],
            [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0],
            [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255],
            [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204],
            [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255],
            [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255],
            [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194],
            [102, 255, 0], [92, 0, 255]]

palette = color_palette()

# segmentation을 원본 이미지와 합쳐서 visualize하는 함수입니다.
def visualize_segmentation_map(image, segmentation_map):
  color_segmentation_map = np.zeros((segmentation_map.shape[0], segmentation_map.shape[1], 3), dtype=np.uint8) # height, width, 3

  for label, color in enumerate(palette):
      color_segmentation_map[segmentation_map == label, :] = color

  ground_truth_color_seg = color_segmentation_map[..., ::-1]

  img = np.array(image) * 0.5 + ground_truth_color_seg * 0.5
  img = img.astype(np.uint8)

  plt.figure(figsize=(15, 10))
  plt.axis('off')
  plt.imshow(img)
  plt.show()

In [ ]:
# 원본 이미지와 segmentation map을 합쳐서 봅시다!
visualize_segmentation_map(image, segmentation_map)

segmentation map은 픽셀 하나하나가 class label로 이루어져있습니다. 각각의 숫자가 어떠한 카테고리를 의미하는지 알아봅시다.

In [ ]:
segmentation_map

In [ ]:
from huggingface_hub import hf_hub_download
import json

# huggingface_hub에서 각 레이블에 해당하는 class명을 가지고 옵니다.
repo_id = f"segments/sidewalk-semantic"
filename = "id2label.json"
id2label = json.load(open(hf_hub_download(repo_id, filename, repo_type="dataset"), "r"))
id2label = {int(k):v for k,v in id2label.items()}
id2label

## 2-4. Data Preprocessing
데이터셋을 불러왔으니, 학습을 위해 torch에서 Dataset 모듈을 가져오고, 전처리를 해봅시다.

In [ ]:
import numpy as np
from torch.utils.data import Dataset

# Dataset을 정의합니다.
class ImageSegmentationDataset(Dataset):

    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

        ################################################################################
        # TODO:                                                                        #
        # Complete Image Preprocessing                                                 #
        ################################################################################

        # TODO: Get image and segmentation map from dataset
        original_image = np.array()
        original_segmentation_map = np.array()

        # TODO: Transform the image and segmentation map
        transformed = self.transform(image=, mask=)
        image, segmentation_map = transformed['image'], transformed['mask']

        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        # convert to C, H, W
        image = image.transpose(2,0,1)

        return image, segmentation_map, original_image, original_segmentation_map

albumentation 라이브러리를 통해 data augmentation과 normalization을 진행합니다.

In [ ]:
import albumentations as A

# 학습 데이터셋에 대한 전처리입니다.
train_transform = A.Compose([
    A.LongestMaxSize(max_size=1333), # 이미지 사이즈를 늘려서
    A.RandomCrop(width=512, height=512), # (512, 512)의 사이즈의 이미지로 랜덤 Crop하고
    A.HorizontalFlip(p=0.5), # 0.5의 확률로 좌우 flip을 적용합니다.
    A.Normalize(), # 이미지를 정규화합니다.
])

# 테스트 데이터셋에 대한 전처리입니다.
test_transform = A.Compose([
    A.Resize(width=512, height=512), # (512, 512)의 사이즈를 가지도록 리사이즈합니다.
    A.Normalize(), # 이미지를 정규화합니다.
])

# 학습 데이터셋과 테스트 데이터셋을 정의합니다.
train_dataset = ImageSegmentationDataset(train_ds, transform=train_transform)
test_dataset = ImageSegmentationDataset(test_ds, transform=test_transform)

################################################################################
# TODO:                                                                        #
# Answer The Question                                                          #
################################################################################
"""
Q: train_transform과 test_transform의 모습이 조금 다르네요. 그 이유는 무엇일까요?

A: (자유롭게 답변해주세요)
"""
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

허깅페이스에서는 모델의 전처리기를 따로 제공해줍니다.

위에서 했던 전처리가 이미지에 대한 일반적인 처리였다면, 아래의 전처리는 오직 MaskFormer만을 위한 전처리입니다.

In [ ]:
from transformers import MaskFormerImageProcessor

# MaskFormer가 이미지를 받을 수 있도록 도와주는 전처리기를 허깅페이스에서 불러옵니다.
preprocessor = MaskFormerImageProcessor(ignore_index=0, reduce_labels=False, do_resize=False, do_rescale=False, do_normalize=False)

In [ ]:
from torch.utils.data import DataLoader
import torch

device = 'cpu'

# 모델에 들어가기 직전 전처리를 해주는 함수입니다.
def collate_fn(batch):
    inputs = list(zip(*batch))
    images = inputs[0]
    segmentation_maps = inputs[1]
    batch = preprocessor(
        images,
        segmentation_maps=segmentation_maps,
        return_tensors="pt",
    )
    batch['pixel_values'] = batch['pixel_values'].to(device)
    batch['class_labels'] = [data.to(device) for data in batch['class_labels']]
    batch['mask_labels'] = [data.to(device) for data in batch['mask_labels']]

    batch["original_images"] = inputs[2]
    batch["original_segmentation_maps"] = inputs[3]

    return batch

# 학습 데이터로더와 테스트 데이터로더를 정의합니다.
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

################################################################################
# TODO:                                                                        #
# Answer The Question                                                          #
################################################################################
"""
Q: train_dataloader는 shuffle=True, test_dataloader는 shuffle=False 입니다.
   그 이유는 무엇일까요?

A: (자유롭게 답변해주세요)
"""
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## 2-5. MaskFormer
허깅페이스에서 pretrained MaskFormer를 가져옵니다.

허깅페이스의 MaskFormer에 대해 궁금하다면 아래 링크를 참조해주세요!

https://huggingface.co/facebook/maskformer-swin-base-ade

<img src='https://drive.google.com/uc?id=148psUqNeMVtjn6BwkfMHSDzBo3wnmEGz'>

MaskFormer는 크게 3개의 모듈로 구성됩니다.

Pixel-Level Module: 이미지에서 feature를 뽑고, transformer module과 segmentation module에 들어갈 임베딩을 뽑습니다.

Transformer Module: DETR과 유사한 구조입니다. N개의 learnable한 쿼리를 image feature와 attention하여 또다른 feature를 뽑아냅니다.

Segmentation Module: MaskFormer의 핵심입니다. 픽셀별로 클래스를 분류했던 기존의 segmentation 모델들과는 달리, class와 mask를 따로 구하는 구조를 가지고 있습니다.

따로 구현은 하지 않겠지만, 데이터 전처리와 학습 과정을 보며 인풋과 아웃풋을 확인해보시기 바랍니다!

In [ ]:
from transformers import MaskFormerForInstanceSegmentation

# 허깅페이스에서 사전 학습된 MaskFormer 모델을 가져옵니다!
model = MaskFormerForInstanceSegmentation.from_pretrained("facebook/maskformer-swin-base-ade",
                                                          id2label=id2label,
                                                          ignore_mismatched_sizes=True)

In [ ]:
batch = next(iter(train_dataloader))
batch.keys()

In [ ]:
################################################################################
# TODO:                                                                        #
# Define Input                                                                 #
################################################################################

# TODO: Read data from batch
outputs = model(pixel_values=,
                class_labels=,
                mask_labels=)

################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

outputs.loss

## 2-6. Training & Evaluation
이제 데이터셋과 모델이 준비되었으니, 학습과 평가를 진행할 차례입니다.

평가 지표는 mean_iou로 설정하였습니다. mean_iou가 궁금하시다면 구글링!

또다시 허깅페이스에서 mean iou를 계산할 수 있는 패키지를 가져옵니다!



In [ ]:
import evaluate

# 평가지표는 mean_iou입니다!
metric = evaluate.load("mean_iou")

In [ ]:
# 하이퍼파라미터를 설정합니다. (1에폭에 약 5~6분 정도 걸리니 참고하시기 바랍니다!)
epochs = 2
learning_rate = 5e-5

In [ ]:
import torch
from tqdm.auto import tqdm

# 모델을 GPU로 보냅니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

running_loss = 0.0
num_samples = 0

for epoch in range(epochs):

  # 학습을 시작합니다.
  print("Epoch:", epoch)
  model.train()
  for idx, batch in enumerate(tqdm(train_dataloader)):

      # 파라미터의 gradient를 초기화합니다.
      optimizer.zero_grad()

      ################################################################################
      # TODO:                                                                        #
      # Define Input                                                                 #
      ################################################################################

      # 추론을 수행합니다.
      # TODO: Read data from batch
      outputs = model(
          pixel_values=,
          mask_labels=,
          class_labels=,
      )

      ################################################################################
      #                                 END OF YOUR CODE                             #
      ################################################################################



      ################################################################################
      # TODO:                                                                        #
      # BackPropagation                                                              #
      ################################################################################

      # TODO: loss를 outputs에서 가져와 역전파를 수행
      loss =
      loss.
      optimizer.

      ################################################################################
      #                                 END OF YOUR CODE                             #
      ################################################################################

      batch_size = batch["pixel_values"].size(0)
      running_loss += loss.item()
      num_samples += batch_size

      if idx % 100 == 0:
        print("Loss:", running_loss/num_samples)

  # 평가를 시작합니다.
  model.eval()
  for idx, batch in enumerate(tqdm(test_dataloader)):

    pixel_values = batch["pixel_values"]

    # 추론을 수행합니다. 테스트 과정에서는 gradient가 기록되지 않습니다.
    with torch.no_grad():
      outputs = model(pixel_values=pixel_values.to(device))

    original_images = batch["original_images"]
    target_sizes = [(image.shape[0], image.shape[1]) for image in original_images]

    # 모델의 아웃풋을 segmentation map으로 만들어 줍니다.
    predicted_segmentation_maps = preprocessor.post_process_semantic_segmentation(outputs,
                                                                                  target_sizes=target_sizes)

    ground_truth_segmentation_maps = batch["original_segmentation_maps"]

    metric.add_batch(references=ground_truth_segmentation_maps, predictions=predicted_segmentation_maps)

  print("Mean IoU:", metric.compute(num_labels = len(id2label), ignore_index = 0)['mean_iou'])

## 2-7. Inference
학습을 완료하셨나요? 모델이 얼마나 잘 배웠는지 눈으로 한 번 봅시다.

In [ ]:
# inference를 해봅시다!
batch = next(iter(test_dataloader))
with torch.no_grad():
  outputs = model(batch["pixel_values"].to(device))

In [ ]:
original_images = batch["original_images"]
target_sizes = [(image.shape[0], image.shape[1]) for image in original_images]
predicted_segmentation_maps = preprocessor.post_process_semantic_segmentation(outputs, target_sizes=target_sizes)

In [ ]:
from PIL import Image
image = batch["original_images"][0]
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 정답 레이블을 visualize 합니다.
gt = batch["original_segmentation_maps"][0]
visualize_segmentation_map(image, gt)

# 모델의 추론 결과를 visualize 합니다.
pred = predicted_segmentation_maps[0].cpu().numpy()
visualize_segmentation_map(image, pred)


################################################################################
# TODO:                                                                        #
# Answer The Question                                                          #
################################################################################
"""
Q: 결과가 어떤가요?
   결과물을 분석하고, 그렇게 나온 이유와 개선방안을 자유롭게 설명해주세요.

A: (자유롭게 답변해주세요)
"""
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################